# YOLOv2 on PYNQ-Z1 and Movidius NCS: HDMI example
To run this notebook, you need to connect the PYNQ-Z1 HDMI input to a video source and connect the HDMI output to a monitor.
### Load required packages and the base overlay

In [9]:
import cv2
import numpy as np
import time
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

import sys
sys.path.insert(0, '../detectionExample')

from ObjectWrapper import *
from Visualize import *

# Load the base overlay
base = BaseOverlay("base.bit")

### Configure the HDMI input and output
We configure both interfaces for RGB as our YOLO network needs RGB frames

In [10]:
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure the HDMI output to the same resolution as the HDMI input
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

# Start the HDMI interfaces
hdmi_in.start()
hdmi_out.start()

### Open the Movidius NCS

In [11]:
# Open the Movidius NCS device
detector = ObjectWrapper('../graph')

Graph file loaded to Movidius NCS


### Calculate subframe indices to cut out of the input frame for the YOLO network

In [12]:
# The input frame width and height
frame_in_width = hdmi_in.mode.width
frame_in_height = hdmi_in.mode.height

# The frame width and height required by the YOLO network
yolo_width = 416
yolo_height = 416

# The subframe indicies to cut-out for the YOLO network (the middle of the input frame)
frame_top = (frame_in_height-yolo_height)//2
frame_bot = frame_in_height-frame_top
frame_left = (frame_in_width-yolo_width)//2
frame_right = frame_in_width-frame_left

### HDMI video pass-through (without YOLO)

In [5]:
n_frames = 2000

start_time = time.time()

for _ in range(n_frames):
    frame = hdmi_in.readframe()
    frame_out = hdmi_out.newframe()
    frame_out[frame_top:frame_bot,frame_left:frame_right,:] = frame[frame_top:frame_bot,frame_left:frame_right,:]
    hdmi_out.writeframe(frame_out)

end_time = time.time()

print('Runtime:',end_time-start_time,'FPS:',n_frames/(end_time-start_time))

Runtime: 38.552093744277954 FPS: 51.87785683616334


### HDMI video with YOLO

In [5]:
n_frames = 200

start_time = time.time()

for _ in range(n_frames):
    # Get a frame from HDMI input
    frame = hdmi_in.readframe()
    
    # Cut out a sub-frame for NCS (416x416) and divide all RGB values by 255
    small_frame = frame[frame_top:frame_bot,frame_left:frame_right,:]
    ncs_frame = small_frame.copy()/255.0

    # Send the frame to the NCS
    results = detector.Detect(frame)
    
    # Interpret results and draw the boxes on the image
    img_res = Visualize(frame, results)

    # Copy labelled image into output frame
    frame_out = hdmi_out.newframe()
    frame_out[frame_top:frame_bot,frame_left:frame_right,:] = img_res[:,:,:]
    hdmi_out.writeframe(frame_out)

end_time = time.time()

print('Runtime:',end_time-start_time,'FPS:',n_frames/(end_time-start_time))

Runtime: 69.28905129432678 FPS: 2.8864589175920137


### HDMI video with YOLO full frame
In this test we resize the whole input frame to the size required by the YOLO network (448x448). The resize operation is performed by software, so it has a significant drag on the runtime and FPS. In the future, I'll try moving the resize operation to the FPGA.

In [13]:
n_frames = 200

start_time = time.time()

for _ in range(n_frames):
    # Get a frame from HDMI input
    frame = hdmi_in.readframe()
    
    # Resize to the frame size required by YOLO network (416x416)
    small_frame = cv2.resize(frame, dsize=(yolo_height, yolo_width), interpolation=cv2.INTER_CUBIC)
    ncs_frame = small_frame.copy()/255.0
    
    # Send the frame to the NCS
    results = detector.Detect(frame)
    
    # Interpret results and draw the boxes on the image
    img_res = Visualize(frame, results)
    
    # Copy labelled image into the output frame
    frame_out = hdmi_out.newframe()
    frame_out[:,:,:] = img_res[:,:,:]
    hdmi_out.writeframe(frame_out)

end_time = time.time()

print('Runtime:',end_time-start_time,'FPS:',n_frames/(end_time-start_time))

Runtime: 136.42510652542114 FPS: 1.4660058188243559


### Close the Movidius NCS

In [14]:
del detector

### Close the HDMI interfaces

In [15]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out